<a href="https://colab.research.google.com/github/leonarddocchagas/data_science/blob/master/semaforos_do_recife.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>